In [4]:
f_func <- "../functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))
f_path <- "../dirs.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_path, '--to=script', '--stdout'), stdout = TRUE)))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

In [5]:
dbs <- c('RS','EPRSFv46','EPRSFv46DM','EPRSc2','EPRSFv46MCAspDM','EPRSFv64','EPRSFv64DM','EPRSFv64MCAspDM')

In [ ]:
n6 <- c("perc", "cnt_cum_taxa", "cnt_taxa", "taxLvl", "taxID", "taxName")
n8 <- c("perc", "cnt_cum_taxa", "cnt_taxa", "miniz", "dist_miniz", "taxLvl", "taxID", "taxName")

lst_perc_root = lst_perc_Fungi = lst_perc_Asp = list()
lst_perc_Asp_species = lst_perc_Asp_species_sumFP = lst_perc_Asp_species_maxFP <- list() 
lst_cnt_Fungi = lst_cnt_Asp = lst_cnt_Asp_species <- list()
    
for (conf_thr in sprintf("%1.1f",c(seq(0.0,1.0,0.1)))){
    sim_nm <- c()
    for (file in list.files(SIM_K2_REP, pattern = paste0("_conf", conf_thr, ".report"))){
        sim_nm <- c(sim_nm, gsub(gsub(c(unlist(strsplit(file, '_'))[1]), 
                    pattern = "^sample1", replacement = ""),
                    pattern = "Syn1", replacement = ""))
    }
    mt <- matrix(ncol = length(unique(sim_nm)), nrow = length(dbs))
    colnames(mt) <- unique(sim_nm)
    rownames(mt) <- rownames(mt) <- gsub(dbs, pattern = "RS_minusT2T", replacement = "RSminusT2T")

    perc_root = perc_Fungi = perc_Asp = perc_Asp_species = perc_Asp_species_sumFP = perc_Asp_species_maxFP <- mt  
    cnt_Fungi = cnt_Asp = cnt_Asp_species <- mt
    
    for (file in list.files(SIM_K2_REP, pattern = paste0("_conf", conf_thr, ".report"))){
        print(file)
        db <- as.factor(unlist(strsplit(gsub(file, pattern = "RS_minusT2T", replacement = "RSminusT2T"), "_"))[2])
        rowN <- which(rownames(mt) == db)
        if(db %nin% rownames(mt)){
            print(db)
            next
        }

        strain <- gsub(gsub(c(unlist(strsplit(file, '_'))[1]), 
                    pattern = "^sample1", replacement = ""),
                    pattern = "Syn1", replacement = "")
        k2rep <- read.csv(file = paste0(SIM_K2_REP, file), header=FALSE, sep = "\t", stringsAsFactors=FALSE)
        
        if (ncol(k2rep) == 6){colnames(k2rep) <- n6}
        if (ncol(k2rep) == 8){colnames(k2rep) <- n8}
    
        k2rep$taxName <- str_trim(k2rep$taxName, side = "left")
        
        
        perc_root[rowN, strain] <- k2rep[which(k2rep$taxName == "root"), "perc"]
        perc_Fungi[rowN, strain] <- k2rep[which(k2rep$taxName == "Fungi"), "perc"]
        perc_Asp[rowN, strain] <- k2rep[which(k2rep$taxName == "Aspergillus"), "perc"]
        
        cnt_Fungi[rowN, strain] <- k2rep[which(k2rep$taxName == "Fungi"), "cnt_cum_taxa"]
        cnt_Asp[rowN, strain] <- k2rep[which(k2rep$taxName == "Aspergillus"), "cnt_cum_taxa"]
        
        species <- gsub(gsub(gsub(strain, pattern = "(?<=[a-z])([A-Z]).*", replacement = "", perl = TRUE),
                        pattern = "costaricaensis", replacement = "costaricensis"), 
                        pattern = "Aspergillus", replacement = "Aspergillus ")
        if (species %in% k2rep$taxName){
            perc_Asp_species[rowN, strain] <- k2rep[which(k2rep$taxName == species), "perc"] 
            cnt_Asp_species[rowN, strain] <- k2rep[which(k2rep$taxName == species), "cnt_cum_taxa"]
        }

        nr_reads_max_any_Asp_species <- k2rep %>% filter(grepl(taxName, pattern = "Aspergillus [a-z]")) %>% 
            filter(taxName != species) %>% 
            filter(!grepl(taxName, pattern = "virus")) %>% filter(taxLvl == "S")  %>% pull(2) %>% max(na.rm = TRUE)
        nr_reads_sum_any_Asp_species <- k2rep %>% filter(grepl(taxName, pattern = "Aspergillus [a-z]")) %>% 
            filter(taxName != species) %>% 
            filter(!grepl(taxName, pattern = "virus")) %>% filter(taxLvl == "S")  %>% pull(2) %>% sum(na.rm = TRUE)
        nr_reads_correct_Asp_species <- k2rep %>% filter(taxName == species) %>% pull(2) %>% sum(na.rm = TRUE)
        nr_reads <- k2rep %>% filter(taxName %in% c("root", "unclassified")) %>% pull(2) %>% sum(na.rm = TRUE)
        perc_Asp_species_maxFP[rowN, strain] <- (nr_reads_max_any_Asp_species/nr_reads)*100 
        perc_Asp_species_sumFP[rowN, strain] <- (nr_reads_sum_any_Asp_species/nr_reads)*100 
        #rm(k2rep)
    }
    lst_perc_root[[paste0("conf", conf_thr)]] <- perc_root
    lst_perc_Fungi[[paste0("conf", conf_thr)]] <- perc_Fungi
    lst_perc_Asp[[paste0("conf", conf_thr)]] <- perc_Asp
    
    lst_cnt_Fungi[[paste0("conf", conf_thr)]] <- cnt_Fungi
    lst_cnt_Asp[[paste0("conf", conf_thr)]] <- cnt_Asp
    lst_cnt_Asp_species[[paste0("conf", conf_thr)]] <- cnt_Asp_species
    
    lst_perc_Asp_species[[paste0("conf", conf_thr)]] <- perc_Asp_species
    lst_perc_Asp_species_sumFP[[paste0("conf", conf_thr)]] <- perc_Asp_species_sumFP
    lst_perc_Asp_species_maxFP[[paste0("conf", conf_thr)]] <- perc_Asp_species_maxFP
}
saveRDS(lst_perc_root,       paste0(INT_ReSeq, "lst_perc_root", ".Rdata"))
saveRDS(lst_perc_Fungi,      paste0(INT_ReSeq, "lst_perc_Fungi", ".Rdata"))
saveRDS(lst_perc_Asp,        paste0(INT_ReSeq, "lst_perc_Asp", ".Rdata"))

saveRDS(lst_cnt_Fungi,       paste0(INT_ReSeq, "lst_cnt_Fungi", ".Rdata"))
saveRDS(lst_cnt_Asp,         paste0(INT_ReSeq, "lst_cnt_Asp", ".Rdata"))
saveRDS(lst_cnt_Asp_species, paste0(INT_ReSeq, "lst_cnt_Asp_species", ".Rdata"))

saveRDS(lst_perc_Asp_species,          paste0(INT_ReSeq, "lst_perc_Asp_species", ".Rdata"))
saveRDS(lst_perc_Asp_species_sumFP,    paste0(INT_ReSeq, "lst_perc_Asp_species_sumFP", ".Rdata"))
saveRDS(lst_perc_Asp_species_maxFP,    paste0(INT_ReSeq, "lst_perc_Asp_species_maxFP", ".Rdata"))

In [ ]:
if (exists("classification_table")){rm("classification_table")}
for (file in list.files(SIM_K2_REP)){
    db <- as.character(unlist(strsplit(gsub(file, pattern = "RS_minusT2T", replacement = "RSminusT2T"), "_"))[2])
        
    if(db %nin% rownames(mt)){
        print(db)
        next
    }
    
    strain <- gsub(gsub(c(unlist(strsplit(file, '_'))[1]), 
                    pattern = "sample1", replacement = ""),
                    pattern = "Syn1", replacement = "")
        
    trshld <- gsub(".report", "", gsub(".*\\_", "", file))

    k2rep <- read.csv(file = paste0(SIM_K2_REP,file), header=FALSE, sep = "\t", stringsAsFactors=FALSE)
    if (ncol(k2rep) == 6){colnames(k2rep) <- n6}
    if (ncol(k2rep) == 8){colnames(k2rep) <- n8}
    k2rep$taxName <- str_trim(k2rep$taxName, side = "left")

    ##TPR Aspergillus genus
    TPR_Asp_g <- k2rep[which(k2rep$taxName == "Aspergillus"), "perc"]

    #TPR Aspergillus species
    species <- gsub(gsub(gsub(strain, pattern = "(?<=[a-z])([A-Z]).*", replacement = "", perl = TRUE),
                        pattern = "costaricaensis", replacement = "costaricensis"), 
                        pattern = "Aspergillus", replacement = "Aspergillus ")
    if (species %in% k2rep$taxName){
        TPR_Asp_s <- k2rep[which(k2rep$taxName == species), "perc"]        
    } else {TPR_Asp_s <- NA}

    ##FPR Aspergillus species
    nr_reads_sum_any_Asp_species <- k2rep %>% filter(grepl(taxName, pattern = "Aspergillus [a-z]")) %>% 
        filter(taxName != species) %>%
        filter(!grepl(taxName, pattern = "virus")) %>% filter(taxLvl == "S")  %>% pull(2) %>% sum(na.rm = TRUE)
    nr_reads_correct_Asp_species <- k2rep %>% filter(taxName == species) %>% pull(2) %>% sum(na.rm = TRUE)
    nr_reads <- k2rep %>% filter(taxName %in% c("root", "unclassified")) %>% pull(2) %>% sum(na.rm = TRUE)
    if (!is.na(nr_reads_correct_Asp_species)){
        FPR_Asp_s <- ((nr_reads_sum_any_Asp_species)/nr_reads)*100        
    } else {
        FPR_Asp_s <- ((nr_reads_sum_any_Asp_species)/nr_reads)*100     
    }

    ##F-score
    F1_score <- TPR_Asp_s/(TPR_Asp_s+(0.5*(FPR_Asp_s + (100-TPR_Asp_g))))*100

    ##yes/no in DB
    if (species %in% k2rep$taxName){in_DB <- "yes"} else {in_DB <- "no"}

    new_rows <- rbind( 
            c(db, strain, trshld, "TP genus", TPR_Asp_g, in_DB),
            c(db, strain, trshld, "TP species", TPR_Asp_s, in_DB),
            c(db, strain, trshld, "FP species", FPR_Asp_s, in_DB),
            c(db, strain, trshld, "F1 score", F1_score, in_DB))
    if (!exists("classification_table")){ 
        classification_table <- new_rows
        colnames(classification_table) <- c("db", "strain", "threshold", "type", "percentage", "in_DB")
    } else {
        classification_table <- rbind(classification_table, new_rows)
    }
}
write.csv(classification_table, paste0(INT_ReSeq, "classification_table.csv"), row.names = TRUE)